# 여러 문서에서 찾아서 답변하는 챗봇 만들기

In [ ]:
!pip install -q langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.0 MB/s et

In [ ]:
!wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip

--2023-08-22 06:27:17--  https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip [following]
--2023-08-22 06:27:18--  https://raw.githubusercontent.com/chatgpt-kr/chatgpt-api-tutorial/main/ch05/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 835816 (816K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 816.23K  --.-KB/s    in 0.01s   

2023-08-22 06:27:18 (79.4 MB/s) - ‘data.zip’ saved [835816/835816]



In [ ]:
!unzip data

Archive:  data.zip
  inflating: 1.txt                   
  inflating: 10.txt                  
  inflating: 11.txt                  
  inflating: 12.txt                  
  inflating: 13.txt                  
  inflating: 14.txt                  
  inflating: 15.txt                  
  inflating: 16.txt                  
  inflating: 17.txt                  
  inflating: 18.txt                  
  inflating: 19.txt                  
  inflating: 2.txt                   
  inflating: 20.txt                  
  inflating: 21.txt                  
  inflating: 22.txt                  
  inflating: 23.txt                  
  inflating: 24.txt                  
  inflating: 25.txt                  
  inflating: 26.txt                  
  inflating: 27.txt                  
  inflating: 28.txt                  
  inflating: 29.txt                  
  inflating: 3.txt                   
  inflating: 30.txt                  
  inflating: 31.txt                  
  inflating: 32.txt            

## Setting up LangChain

OpenAI API Key

https://platform.openai.com/account/api-keys

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "여러분의 Key값을 사용하세요"

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

## Load multiple and process documents

In [ ]:
loader = DirectoryLoader('.', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()

print('문서의 개수 :', len(documents))

문서의 개수 : 57


In [ ]:
print('1번 문서 :', documents[1])
print('-' * 20)
print('21번 문서 :', documents[21])
print('-' * 20)

1번 문서 : page_content='정책 내용: 서울 영테크\n청년들 중 자산 형성이 어려운 이들을 위해 서울시청 미래청년기획단이 주최하는 금융 정책을 소개합니다. 이 정책은 청년들에게 효과적이고 체계적인 재테크 교육과 맞춤형 상담을 제공하여 경제적 자립을 지원하는 사업입니다.\n\n지원 내용은 대면 및 비대면으로 이루어지는 재무상담과 금융 교육으로 구성됩니다. 대면상담은 합의한 장소에서 진행되며, 비대면상담은 화상, 전화, 이메일을 통해 이루어집니다. 금융 교육은 온라인 교육과 오프라인 교육으로 제공되며, 온라인 교육은 청년 몽땅 정보통 또는 서울청년생활(유튜브)에서 이용할 수 있습니다. 오프라인 교육은 월 2회로 진행되는 서울 영테크 원데이 스쿨에서 제공됩니다.\n\n이 정책은 2023년 4월 3일부터 2023년 12월 31일까지 운영되며, 참여 대상은 서울 거주하고 있는 19세부터 39세까지의 청년들이며, 총 10,000명이 참여할 수 있습니다.\n\n신청은 청년 몽땅 정보통(이전 서울청년포털)을 통해 신청하실 수 있으며, 심사 및 발표는 수시로 진행됩니다. 신청에 관한 문의는 다산콜센터(02-120) 또는 전담콜센터(02-1644-7747)로 전화하시거나, 카카오톡의 서울톡 또는 서울 영테크 플러스 친구를 통해 문의하실 수 있습니다. 이 사업은 서울시청 미래청년기획단이 운영하며, 자세한 내용은 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n청년금융기술지원센터 소개: https://youth.seoul.go.kr/site/main/content/youth_young_tech_intro\n신청사이트: https://youth.seoul.go.kr/site/main/youngTech/selfCheck' metadata={'source': '34.txt'}
--------------------
21번 문서 : page_content='정책 이름: 서울미래인재 육성 및 활동 지원\n서울시청 미래청년기획단에서는 사회참여를 촉진하기 위해 서울시 위원회

## Split texts

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수 :', len(texts))

분할된 텍스트의 개수 : 64


In [ ]:
texts[0]

Document(page_content='정책제목: 서울시 청년허브 운영\n서울시청 미래청년기획단에서 주최하는 사회참여 정책은 청년활동 생태계를 조성하여 청년활동의 지속성과 정책 효과성을 향상시키고, 청년 사회참여 및 국내외 청년 네트워크를 활성화하는 것을 목표로 합니다. 이 정책은 지속가능한 서울을 위한 청년정책 개발 및 청년활동생태계 구축, 실험과 시도를 통한 청년의 창의적 생산활동 지원, 세대와 지역을 넘는 교류협력을 통한 청년자산 마련을 지원합니다.\n\n이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청자격은 19세부터 39세까지의 청년디자인창업기업입니다. 학력, 전공, 취업상태, 특화분야에 대한 요건은 없으며, 지원규모 및 관련사이트에 대한 정보는 제공되지 않았습니다.\n\n신청은 https://youthhub.kr/에서 신청할 수 있으며, 심사 및 발표에 대한 정보는 공지되지 않았습니다. 자세한 제출서류에 대한 정보는 제공되지 않았습니다.\n\n이 정책은 (사단법인)씨즈에서 운영됩니다. 참고로 https://youthhub.kr/ 사이트를 참고하시기 바랍니다.', metadata={'source': '11.txt'})

In [ ]:
source_lst = []
for i in range(0, len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서 :', filtered_counts)
print('분할된 텍스트의 개수 :', len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'22.txt': 2, '36.txt': 2, '49.txt': 2, '23.txt': 2, '40.txt': 2, '31.txt': 2, '48.txt': 2}
분할된 텍스트의 개수 : 64


## Create Chroma DB

In [ ]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding)

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["source"])

유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': '23.txt'}
----------------------------------------
각 유사 문서의 문서 출처 :
23.txt
23.txt
40.txt
39.txt


### 결과를 k개 반환

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
docs = retriever.get_relevant_documents("신혼 부부를 위한 정책이 있어?")

for doc in docs:
    print(doc.metadata["source"])

23.txt
23.txt


## Make a chain


```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Query

In [ ]:
input_text = "대출과 관련된 정책이 궁금합니다"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '대출과 관련된 정책이 궁금합니다', 'result': '서울시 전략주택공급과에서는 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차보증금 대출을 지원하는 정책을 운영하고 있습니다. 이 정책은 혼인 감소 및 출산율 감소 문제를 해결하기 위한 것으로, 임차보증금에 대해 무이자 지원을 제공합니다.\n\n지원 내용은 다음과 같습니다:\n- 임차보증금이 1억원을 초과하는 경우: 보증금의 30% 지원 (청년 최대 4,500만원, 신혼부부 최대 6,000만원)\n- 임차보증금이 1억원 이하인 경우: 보증금의 50% 지원 (청년 및 신혼부부 최대 4,500만원)\n\n신청 자격은 19세에서 39세까지이며, 임차보증금 무이자 지원을 받을 수 있는 대상입니다. 학력 전공, 취업 상태 등 제한은 없습니다. 다만, 서울시 임차보증금 및 이자 지원과 중복 신청은 불가능하며, 타 금융기관의 전세자금 대출과는 중복 가능하나 대출 가능 여부는 해당 금융기관에 문의해야 합니다.\n\n신청은 신청서 및 필요한 서류를 제출한 후 대상자가 선정되고 통보됩니다. 이후 임대차 계약 체결과 주거비지원 약정서 체결이 이루어진 후, 지원금액이 임대인 계좌로 입금됩니다.\n\n자세한 신청 방법과 필요한 서류는 서울시 소셜공동체 온라인 포털에서 확인하실 수 있습니다. 추가적인 사항은 참고사이트1를 참고하시기 바랍니다.', 'source_documents': [Document(page_content='정책제목: 역세권청년주택 주거비지원\n서울시 전략주택공급과에서는 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차보증금 대출을 지원하는 정책을 운영하고 있습니다. 이 정책은 혼인 감소 및 출산율 감소 문제를 해결하기 위한 것으로, 임차보증금에 대해 무이자 지원을 제공합니다.\n\n지원 내용은 다음과 같습니다:\n임차보증금이 1억원을 초과하는 경우: 보증금의 30% 지원 (청년 최대 4,500만원, 신혼부부 최대 6,000만원)\n임차보증금이 1억원 이하인 경우: 보증금의 50% 지원 (청년

In [ ]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['source'])

"신혼 부부의 신혼집 마련을 위한 정책이 있을까?"라는 임의의 입력에 대해서 챗봇의 답변을 확인해보겠습니다.

In [ ]:
input_text = "신혼 부부의 신혼집 마련을 위한 정책이 있을까?"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 신혼 부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
23.txt
23.txt


In [ ]:
# print(chatbot_response)
chatbot_response

{'query': '신혼 부부의 신혼집 마련을 위한 정책이 있을까?',
 'result': '네, 신혼 부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서는 "신혼부부 임차보증금 지원" 정책을 운영하고 있습니다. 이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.',
 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata

In [ ]:
input_text = "전세자금이 부족한 사람을 위한 정책 이름이 뭐야?"
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

정책 이름은 "희망두배 청년통장"입니다.

문서 출처:
40.txt
40.txt


In [ ]:
query = "희망두배 청년통장은 어떤걸 지원하니?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장은 저소득 근로 청년들의 자립을 지원하기 위해 서울시에서 운영하는 정책입니다. 이 정책은 교육비, 주거비, 결혼자금, 창업운영자금 등을 위한 저축을 지원하는 내용을 포함하고 있습니다. 신청자격은 18세에서 34세까지이며, 서울시 거주자이고 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우 신청할 수 있습니다. 자세한 내용과 신청 방법은 서울시복지재단의 홈페이지 및 서울시 안심돌봄복지과의 홈페이지에서 확인하실 수 있습니다.

문서 출처:
40.txt
36.txt


In [ ]:
query = "희망두배 청년통장의 지원 조건은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 지원 조건은 다음과 같습니다:

1. 신청자격: 18세에서 34세까지의 청년이며, 서울시 거주자이고 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우 신청할 수 있습니다.

2. 신청 방법: 방문접수, 우편 신청(도착분 기준 18:00까지 동주민센터 도착), 이메일 신청(접수 마감일 18:00까지 도착) 방법으로 신청할 수 있습니다. 이메일로 제출할 경우 원본 스캔 또는 이미지 촬영하여 PDF, JPG, PNG 파일 형식으로 제출해야 합니다.

3. 신청 기한: 신청 기한 내에 제출서류가 누락되거나 식별이 불가능한 경우 대상자 선정에서 제외될 수 있습니다.

4. 선정 결과: 선정 결과는 추후에 공지될 예정입니다.

5. 필요한 서류: 자세한 신청 방법과 필요한 서류는 서울시복지재단, 서울시 및 25개 자치구 홈페이지에서 확인하실 수 있습니다.

기타 문의 사항은 1688-1453, 120다산콜, 해당 주소지의 자치구 및 동주민센터로 문의하시면 됩니다.

문서 출처:
40.txt
36.txt


In [ ]:
query = "희망두배 청년통장의 마감 기한은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 마감 기한은 문맥에서 명확하게 언급되지 않았습니다. 따라서 정확한 마감 기한을 알 수 없습니다. 자세한 내용은 서울시복지재단의 홈페이지나 서울시 안심돌봄복지과의 홈페이지에서 확인하실 수 있습니다.

문서 출처:
40.txt
37.txt


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.2 MB/s eta 0:00:00
   

To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [ ]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>